In [1]:
import pandas as pd
import re
import os
from PyPDF2 import PdfReader  # type: ignore
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from joblib import dump, load

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_file):
    with open(pdf_file, 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

# Read the CSV file containing resume data
df = pd.read_csv("C:/Users/Kainat Ahmed/Desktop/Resume-Screener/data.csv")

# Define a function to clean the text
def clean_text(text):
    # Remove specific patterns from text
    cleaned_text = re.sub(r'Page\s+\d+\s+of\s+\d+|Confidential|Header|Footer', '', text, flags=re.IGNORECASE)
    # Convert text to lowercase
    cleaned_text = cleaned_text.lower()
    # Remove all non-alphanumeric characters except whitespace
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)
    return cleaned_text

# Clean the 'Resume' text data
df['Cleaned_Resume'] = df['Resume'].apply(clean_text)

# Handling missing values
imputer = SimpleImputer(strategy='most_frequent')
df['Cleaned_Resume'] = imputer.fit_transform(df[['Cleaned_Resume']])[:, 0]

# Split the data into features (X) and labels (y)
X = df[['Cleaned_Resume']]
y = df['Category']  # Assuming 'Category' contains the labels

# Define the column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('tfidf', TfidfVectorizer(max_features=1000), 'Cleaned_Resume'),
    ],
    remainder='drop'
)

# Define the full pipeline with advanced data processing
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Train the pipeline
pipeline.fit(X, y)

# Save the pipeline to a file
pipeline_file = 'pipeline.joblib'
dump(pipeline, pipeline_file)

# Load the pipeline from the file
loaded_pipeline = load(pipeline_file)

# Function to process PDFs in a directory using the loaded pipeline
def process_pdfs_in_directory(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.pdf'):
            file_path = os.path.join(directory, filename)
            # Extract text from the PDF
            text = extract_text_from_pdf(file_path)
            # Preprocess the text
            cleaned_text = clean_text(text)
            # Handle missing values for new data
            cleaned_text = imputer.transform([[cleaned_text]])[0][0]
            # Predict the category using the loaded pipeline
            category = loaded_pipeline.predict([cleaned_text])[0]
            # Create a directory for the category (job title) if it doesn't exist
            category_dir = os.path.join(directory, str(category))
            if not os.path.exists(category_dir):
                os.makedirs(category_dir)
            # Move the PDF to the corresponding category folder
            new_file_path = os.path.join(category_dir, filename)
            os.rename(file_path, new_file_path)

print("Documents categorized successfully")

# Provide the directory containing PDF files as input
directory = "C:/Users/Kainat Ahmed/Desktop/Resume-Screener/Categorized_PDFs"
process_pdfs_in_directory(directory)

Documents categorized successfully
